# 博客项目

> 博客项目大体实现：  
> - 文章分类显示  
> - 分页  
> - 标签云  
> - 浏览排行  
> - 评论排行  
> - 站长推荐  
> - 评论  
> - 评论展示  

## 创建项目

```python
django-admin startproject blog
```

## 创建应用
```python
python manage.py startapp demo

```

## 配置setings.py

- 搭建应用
```python
INSTALLED_APPS = [
  'django.contrib.admin',
  'django.contrib.auth',
  'django.contrib.contenttypes',
  'django.contrib.sessions',
  'django.contrib.messages',
  'django.contrib.staticfiles',
  'demo',   # 修改点
]
```

- 配置templates模板查找路径
```python
TEMPLATES = [
  {
      'BACKEND': 'django.template.backends.django.DjangoTemplates',
      'DIRS': [os.path.join(BASE_DIR, 'templates')],   # 修改点
      'APP_DIRS': True,
      'OPTIONS': {
          'context_processors': [
              'django.template.context_processors.debug',
              'django.template.context_processors.request',
              'django.contrib.auth.context_processors.auth',
              'django.contrib.messages.context_processors.messages',
          ],
      },
  },
]
```

- 连接MySQL数据库
```python
DATABASES = {
  'default': {
      'ENGINE': 'django.db.backends.mysql',   # 链接MySQL
      'NAME': 'blog_test',             # 数据库名
      'USER': 'root',                 # 用户名
      'PASSWORD': '123456',             # 密码
      'HOST': '127.0.0.1',             # 数据库IP
      'POST': '3306',                # 数据库端口号
  }
}
```

- 修改中文和时区配置  

```python
LANGUAGE_CODE = 'zh-hans'  # 中文字符集设置

TIME_ZONE = 'Asia/Shanghai'  # 时区设置

USE_I18N = True

USE_L10N = True

USE_TZ = False    # 修改
```

- 配置静态文件查找路径
```python
STATICFILES_DIRS = [
  os.path.join(BASE_DIR, 'static')
]
```

- 配置多媒体文件查找路径
```python
MEDIA_ROOT = os.path.join(BASE_DIR, 'static/media')
```

## 配置主路由

```python
from django.conf.urls import url, include
from django.contrib import admin

urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'', include('demo.urls'))
]
```

## 创建模板、静态、多媒体文件

三个目录均是根据`settings.py`中设置查找路径进行创建的

在项目目录下创建模板存放目录`templates`,用于存放HTML模板

在项目目录下创建静态文件存放目录`static`,用于存放js、css、img等渲染文件

在项目目录`static`下创建多媒体文件存放目录`media`，用于存放多媒体文件，如图片，视频或音频等


-----------------------------

最后并将HTML、static全部拖拽到项目中相应位置，导入前端显示页面

## 配置子路由

```python
from django.conf.urls import url
from .views import *

urlpatterns = [
    url(r'^$', index),
    # 路由连接博客主页
    url(r'^category/(\d+)/$', category),
    # 分类页
    url(r'^article/(\d+)/$', article),
    # 文章详情页
    url(r'^comment/(\d+)/$', comment),
    # 发表评论
]
```

## 配置模型类models

```python
from django.db import models
from django.contrib.auth.models import User
# Create your models here.


# 分类表
class Category(models.Model):
    name = models.CharField(max_length=20, verbose_name="分类名")

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = "分类表"
        verbose_name_plural = verbose_name


# 用户表
class UserProfiles(models.Model):
    # 用户名与django中user表一对一关联
    user = models.OneToOneField(User, verbose_name="用户名")
    nick_name = models.CharField(max_length=50, verbose_name="昵称")
    qq = models.CharField(max_length=20, verbose_name="QQ号")
    user_img = models.ImageField(upload_to="user_img", verbose_name="头像")

    def __str__(self):
        return self.nick_name

    class Meta:
        verbose_name = "用户表"
        verbose_name_plural = verbose_name


# 文章表
class Article(models.Model):
    title = models.CharField(max_length=50, verbose_name="标题")
    author = models.ForeignKey(UserProfiles, verbose_name="作者")
    category = models.ForeignKey(Category, verbose_name="分类")
    publish_time = models.DateTimeField(auto_now_add=True, verbose_name="发布时间")
    intro = models.CharField(max_length=50, verbose_name="简介")
    content = models.TextField(max_length=1000, verbose_name="内容")
    article_pic = models.ImageField(upload_to="article_pic", verbose_name="文章图片")
    page_views = models.IntegerField(default=0, verbose_name="浏览量")
    recommend = models.BooleanField(default=True, verbose_name="是否推荐")

    def __str__(self):
        return self.title

    class Meta:
        verbose_name = "文章表"
        verbose_name_plural = verbose_name


# 标签云
class Label(models.Model):
    name = models.CharField(max_length=32)

    def __str__(self):
        return self.name

    class Meta:
        verbose_name = "标签表"
        verbose_name_plural = verbose_name


# 评论表
class Comment(models.Model):
    article = models.ForeignKey(Article, verbose_name="评论文章")
    comment_date = models.DateTimeField(auto_now_add=True, verbose_name="评论日期")
    content = models.TextField(max_length=512, verbose_name="评论内容")

    def __str__(self):
        return self.article

    class Meta:
        verbose_name = "评论表"
        verbose_name_plural = verbose_name

```

## 注册后台管理

```python
from django.contrib import admin
from .models import *
# Register your models here.


admin.site.register(UserProfiles)    # 注册用户表
admin.site.register(Article)        # 注册文章表
admin.site.register(Category)       # 注册分类表
admin.site.register(Label)         # 注册标签表
admin.site.register(Comment)        # 注册评论表
```


## 配置视图函数views

- 导入模块 

```python
from django.shortcuts import render, redirect
from .models import *
from django.core.paginator import Paginator

```

- index主页

```python
def index(request):
    all_article1 = Article.objects.all()
    paginator = Paginator(all_article1, 1)
    page = request.GET.get('page', 1)
    all_article = paginator.page(page)

    page_sum = paginator.num_pages
    page_list = paginator.page_range
    return render(request, 'index.html', locals())

```

- category分类

```python
def category(request, id):
    category_article = Article.objects.filter(category_id=int(id))
    paginator = Paginator(category_article, 1)
    page = request.GET.get('page', 1)
    all_article = paginator.page(page)

    page_sum = paginator.num_pages
    page_list = paginator.page_range
    return render(request, 'category.html', locals())

```

- article文章详情

```python
# 文章详情页
def article(request, id):
    article_obj = Article.objects.get(pk=int(id))
    # 浏览数+1
    article_obj.page_views += 1
    article_obj.save()
    all_comment = Comment.objects.filter(article=int(id))
    # 获取文章对应的所有评论，渲染到页面
    return render(request, 'article.html', locals())

```

- comment评论

```python
def comment(request, id):
    article_title = Article.objects.get(id=id).title
    # 获取文章标题，渲染到评论页中，说明要评论哪篇文章
    if request.method == 'POST':
        content = request.POST.get('comment')
        Comment.objects.create(
            article_id=id,
            content=content
        )
        url = '/article/' + id
        return redirect(url)
    return render(request, 'comment.html', locals())

```

- Golbal全局函数

```python
# 全局变量：每一个模板都能使用的函数
# 定义全局变量，必须在settings.py中templates里添加过来
def Global(request):
    all_label = Label.objects.all()
    # 获取所有标签
    all_category = Category.objects.all()
    # 获取所有分类
    all_article_views = Article.objects.all().order_by('-page_views')[0:4]
    # 获取浏览排行前四的文章
    all_article_recommend = Article.objects.filter(recommend=True).order_by('-publish_time')[:4]
    # 获取所有推荐的文章,发布时间倒叙排序的前四个
    return locals()

```

> 要将全局函数注册到`templates`里，以便全局引用  
```python
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [os.path.join(BASE_DIR, 'templates')],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
                'demo.views.Global',        # 添加全局变量
            ],
        },
    },
]
```


## 导入HTML等模板

- index.html分页HTML示例

```html
<div id="pagination">
<ul id="pagination-flickr">

    {% if all_article.has_previous %}
        <li class="previous-off"><a href="?page={{ all_article.number|add:-1 }}">&laquo;上一页</a></li>
    {% endif %}

    <li style="font-size: 16px">{{ all_article.number }}/{{ page_sum }}</li>
    
    {% if all_article.has_next %}
      <li class="next"><a href="?page={{ all_article.number|add:1 }}">下一页 &raquo;</a></li>
    {% endif %}
    
</ul>
</div>

```

- 显示分类导航HTML示例

```html
<ul id="head_ul">
     <a class="active" href="/" target="_blank">首页</a>
     {% for category in all_category %}
        <a href="/category/{{ category.id }}">{{ category.name }}</a>
     {% endfor %}

</ul>
```

- 文章显示HTML示例

```html
<div class="topnews">
   
 <h2>最新文章</h2>
    {% for article in all_article %}
       
        <div class="blogs">
        <ul>
          <h3><a href="/article/{{ article.id }}">{{ article.title }}</a></h3>
          <p>{{ article.intro }}</p>
          <p><img src="/static/media/{{ article.article_pic }}" style="height: 50px"></p>
          <p class="autor">
              <span class="lm f_l">
                  <a href="/">{{ article.category }}</a>
              </span>
              <span class="dtime f_l">{{ article.publish_time }}</span>
              <span class="viewnum f_r">浏览（<a href="/">{{ article.page_views }}</a>）</span>
              <span class="pingl f_r">评论(<a href="/comment/{{ article.id }}">{{ article.comment_set.count }}</a>)</span>
          </p>
        </ul>
        </div>
    {% endfor %}
</div>

```

> 注意图片路径！！！

- 浏览排行HTML示例

```html
<ul>
    {% for article in all_article_views %}
      <li><a href="/article/{{ article.id }}/" target="_blank">{{ article.title }}</a></li>
    {% endfor %}

</ul>
```

- 站长推荐HTML示例

```html
<ul>
    {% for article in all_article_recommend %}
        <li><a href="/article/{{ article.id }}/" target="_blank">{{ article.title}}</a></li>
    {% endfor %}
</ul>
```

- 评论推荐HTML示例

```html

```

- 标签云HTML示例

```html
<div class="cloud">
    <h3>标签云</h3>
      <ul>
          {% for label in all_label %}
          <li><a href="#">{{ label.name }}</a></li>
          {% endfor %}

      </ul>
</div>
```

- category.html

```
{% extends 'index.html' %}
```

- article.html评论显示HTML示例

```html
<h3 id="respond"><a href="/comment/{{ article_obj.id }}">请发表评论</a></h3>

<div class="commentstitle">
          <span class="trackback">
              <a href="http://www.baidu.org/wp-trackback.php?p=542" rel="trackback" title="Trackback URI"></a>
          </span>
          <h3 id="comments">
              <span class="commentsnumber">只有 {{ article_obj.comment_set.count }} 评论</span>到目前为止
          </h3>
      </div>
      <ol class="commentlist">
          {% for comment in all_comment %}
           <li id="comment-59419">
                <div class="top">
                    <a href="http://www.yopoing.com" rel="external nofollow" class="url">{{ comment.author }}</a>
                    <span class="time"> @ <a href="#comment-59419" title="">{{ comment.comment_date }}</a></span>
                </div>
                
                <div class="body">
                    <p>{{ comment.content }}</p>
                </div>
              </li>
          {% endfor %}
      </ol>

```

- article.html文章详情页

```html
<div>
     <h2>
         <a href="http://www.baidu.org/?p=542" rel="bookmark">文章: {{ article_obj.title }}</a>
     </h2>
     <div class="postmeta">
         <span class="postmeta_author">作者昵称: {{ article_obj.author.nick_name }}</span>
         <br>
         <span class="postmeta_category"><a href="" rel="category">分类: {{ article_obj.category.name }}</a></span>
         <span class="postmeta_time">创建时间: {{ article_obj.publish_time | date:"Y-m-d" }}</span>
     </div><!-- end postmeta -->
     <div class="entry">
         <img style="height:300px;" src="/static/media/{{ article_obj.article_pic }}" >
         <div style="color:red;"><strong>内容: {{ article_obj.content }}</strong>
     </div>
</div>
```

- comment.html发表评论HTML示例

```html
<form method="post" id="commentform"> 
	{% csrf_token %}
    <p><textarea name="comment" placeholder="评论@{{ article_title }}" id="comment" cols="25" rows="5" tabindex="4" class="message_input" ></textarea></p>
    <p><input name="submit" type="submit" id="submit" tabindex="5" value="提交" class="button" />
    <input type="hidden" name="comment_post_ID" value="542" />
    </p>
</form>
```



## 迁移文件

```python
python manage.py makemigrations
python manage.py migrate
```

## 创建超级管理员

```python
python manage.py createsuperuser
```

## 启动服务

```python
python manage.py runserver
```

